### Setup the connection 

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Myrto.Setzi/Documents/Ironhack/code_alongs/ironhack_service_account_big_query.json"

In [3]:
import six
six.moves.reload_module(six)

<module 'six' from 'C:\\Users\\myrto.setzi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\six.py'>

In [4]:
from google.cloud import bigquery

In [5]:
client = bigquery.Client()

### Challenge 1 - Who Have Published What At Where?

In [5]:
query_1='''
SELECT 
    a.au_id AS author_id
    , au_lname AS last_name
    , au_fname AS first_name
    , t.title AS title
    , p.pub_name AS publisher
FROM  
    `ironhack-data-analytics-265219.publications.authors` a
INNER JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` ta 
ON
    a.au_id=ta.au_id
INNER JOIN 
    `ironhack-data-analytics-265219.publications.titles` t 
ON 
    ta.title_id=t.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.publishers` p 
ON 
    t.pub_id=p.pub_id
'''

In [6]:
query_job_1 = client.query(query=query_1)

In [7]:
df_1=query_job_1.to_dataframe()

In [8]:
df_1.head(10)

,author_id,last_name,first_name,title,publisher
0,807-91-6654,Panteley,Sylvia,"Onions, Leeks, and Garlic: Cooking Secrets of ...",Binnet & Hardley
1,722-51-5454,DeFrance,Michel,The Gourmet Microwave,Binnet & Hardley
2,712-45-1867,del Castillo,Innes,Silicon Valley Gastronomic Treats,Binnet & Hardley
3,899-46-2035,Ringer,Anne,Is Anger the Enemy?,New Moon Books
4,899-46-2035,Ringer,Anne,The Gourmet Microwave,Binnet & Hardley
5,998-72-3567,Ringer,Albert,Is Anger the Enemy?,New Moon Books
6,998-72-3567,Ringer,Albert,Life Without Fear,New Moon Books
7,172-32-1176,White,Johnson,Prolonged Data Deprivation: Four Case Studies,New Moon Books
8,486-29-1786,Locksley,Charlene,Emotional Security: A New Algorithm,New Moon Books
9,486-29-1786,Locksley,Charlene,Net Etiquette,Algodata Infosystems


In [9]:
query_2='''
SELECT 
    COUNT(*) AS vol
FROM  
     `ironhack-data-analytics-265219.publications.titleauthor`
'''

In [10]:
query_job_2 = client.query(query=query_2)

In [11]:
df_2=query_job_2.to_dataframe()
df_2

,vol
0,25


In [12]:
df_1.count()

author_id     25
last_name     25
first_name    25
title         25
publisher     25
dtype: int64

### Challenge 2 - Who Have Published How Many At Where?

In [13]:
query_3='''
SELECT
    a.au_id AS author_id
    , au_lname AS last_name
    , au_fname AS first_name
    , p.pub_name AS publisher
    , COUNT(t.title_id) AS title_count
FROM
    `ironhack-data-analytics-265219.publications.authors` a
INNER JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` ta 
ON
    a.au_id=ta.au_id
INNER JOIN 
    `ironhack-data-analytics-265219.publications.titles` t 
ON
    ta.title_id=t.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.publishers` p 
ON
    t.pub_id=p.pub_id
GROUP BY
    1,2,3,4
    '''

In [14]:
query_job_3 = client.query(query=query_3)

In [15]:
df_3=query_job_3.to_dataframe()
df_3.head(10)

,author_id,last_name,first_name,publisher,title_count
0,807-91-6654,Panteley,Sylvia,Binnet & Hardley,1
1,722-51-5454,DeFrance,Michel,Binnet & Hardley,1
2,712-45-1867,del Castillo,Innes,Binnet & Hardley,1
3,899-46-2035,Ringer,Anne,New Moon Books,1
4,899-46-2035,Ringer,Anne,Binnet & Hardley,1
5,998-72-3567,Ringer,Albert,New Moon Books,2
6,172-32-1176,White,Johnson,New Moon Books,1
7,486-29-1786,Locksley,Charlene,New Moon Books,1
8,486-29-1786,Locksley,Charlene,Algodata Infosystems,1
9,846-92-7186,Hunter,Sheryl,Algodata Infosystems,1


### Challenge 3 - Best Selling Authors

In [16]:
query_4='''
SELECT
    a.au_id AS author_id
    , au_lname AS last_name
    , au_fname AS first_name
    , SUM(s.qty) AS total
FROM
    `ironhack-data-analytics-265219.publications.authors` a
INNER JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` ta 
ON
    ta.au_id = a.au_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.titles` t 
ON
    t.title_id = ta.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.sales` s 
ON
    s.title_id = t.title_id
GROUP BY 
    1,2,3
ORDER BY 
    total DESC
LIMIT 3
'''

In [17]:
query_job_4 = client.query(query=query_4)

In [18]:
df_4=query_job_4.to_dataframe()
df_4

,author_id,last_name,first_name,total
0,899-46-2035,Ringer,Anne,148
1,998-72-3567,Ringer,Albert,133
2,846-92-7186,Hunter,Sheryl,50


### Challenge 4 - Best Selling Authors Ranking

In [19]:
query_5='''
SELECT
    a.au_id AS author_id
    , au_lname AS last_name
    , au_fname AS first_name
    , COALESCE(SUM(s.qty), 0) AS total
FROM
    `ironhack-data-analytics-265219.publications.authors` a
LEFT JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` ta 
ON
    ta.au_id = a.au_id
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titles` t 
ON
    t.title_id = ta.title_id
LEFT JOIN
    `ironhack-data-analytics-265219.publications.sales` s 
ON
    s.title_id = t.title_id
GROUP BY
    1,2,3
ORDER BY
    total DESC;
'''

In [20]:
query_job_5 = client.query(query=query_5)
df_5=query_job_5.to_dataframe()
df_5

,author_id,last_name,first_name,total
0,899-46-2035,Ringer,Anne,148
1,998-72-3567,Ringer,Albert,133
2,846-92-7186,Hunter,Sheryl,50
3,427-17-2319,Dull,Ann,50
4,213-46-8915,Green,Marjorie,50
5,724-80-9391,MacFeather,Stearns,45
6,267-41-2394,O'Leary,Michael,45
7,807-91-6654,Panteley,Sylvia,40
8,722-51-5454,DeFrance,Michel,40
9,238-95-7766,Carson,Cheryl,30


### Bonus Challenge - Most Profiting Authors

In [6]:
query_6='''
WITH royalties_table AS(
SELECT
    title_id
    , au_id
    , au_lname
    , au_fname
    , SUM(advance_au) AS advance
    , SUM(royalties) AS royalties 
FROM 
    (SELECT
        t.title_id
        , t.price
        , (t.advance * ta.royaltyper / 100) AS advance_au
        , t.royalty
        , s.qty
        , a.au_id
        , au_lname
        , au_fname
        , ta.royaltyper
        , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
    FROM 
        `ironhack-data-analytics-265219.publications.titles` t
    INNER JOIN
        `ironhack-data-analytics-265219.publications.sales` s 
    ON
        s.title_id = t.title_id
    INNER JOIN
        `ironhack-data-analytics-265219.publications.titleauthor` ta 
    ON
        ta.title_id = s.title_id
    INNER JOIN
        `ironhack-data-analytics-265219.publications.authors` a 
    ON
        a.au_id = ta.au_id)a
GROUP BY
    1,2,3,4)
            SELECT
                au_id AS author_id
                , au_lname AS last_name
                , au_fname AS first_name
                , sum(advance + royalties) AS profits
            FROM
                royalties_table
            GROUP BY
                1,2,3
            ORDER BY
                profits DESC
            LIMIT 3
'''

In [7]:
query_job_6 = client.query(query=query_6)
df_6=query_job_6.to_dataframe()
df_6

,author_id,last_name,first_name,profits
0,722-51-5454,DeFrance,Michel,22521.528
1,213-46-8915,Green,Marjorie,14162.110
2,899-46-2035,Ringer,Anne,12128.132
